In [1]:
import gerrychain

from gerrychain import Graph, Partition, MarkovChain
from gerrychain.updaters import Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from gerrychain.constraints import contiguous

import networkx as nx
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from functools import partial

In [2]:
graph = Graph.from_file("./data/GA_CD_example/")

/home/peter/Dropbox/MADLAB/Git_Repos/gerrytools-dev/.venv/lib/python3.11/site-packages/gerrychain/graph/graph.py:262: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


In [3]:
for key in graph.nodes[0].keys():
    print(key)

boundary_node
area
ID
DISTRICT
PRECINCT_I
PRECINCT_N
CTYNAME
FIPS1
FIPS2
PRES16D
PRES16R
PRES16L
SEN16D
SEN16R
SEN16L
TOTPOP
NH_WHITE
NH_BLACK
NH_AMIN
NH_ASIAN
NH_NHPI
NH_OTHER
NH_2MORE
HISP
H_WHITE
H_BLACK
H_AMIN
H_ASIAN
H_NHPI
H_OTHER
H_2MORE
VAP
HVAP
WVAP
BVAP
AMINVAP
ASIANVAP
NHPIVAP
OTHERVAP
2MOREVAP
CD
HDIST
SEND
geometry


``` 
  {'APBVAP20': [0.529178733914834,
   0.5835856208817768,
   0.325959841129744,
   0.31219226754668045,
   0.6469278046382038,
   0.47365778499369376],
  'COLVAP20': [0.6284230784310049,
   0.6991969199718455,
   0.37949580759046775,
   0.5391578015119086,
   0.7317167909530444,
   0.5214507671895139],
  'HVAP20': [0.09924434451617083,
   0.11561129909006877,
   0.05353596646072374,
   0.2269655339652282,
   0.08478898631484061,
   0.0477929821958201],
  'WVAP20': [0.31253078959435293,
   0.2709610671551293,
   0.5776337020079435,
   0.3193458939133552,
   0.23671883962959928,
   0.44606898407993334],
  'effectiveness': 5,
  'step': 999}]
```

In [4]:
cds = set({})
for node in graph.nodes:
    cds.add(graph.nodes[node]["CD"])

In [5]:
cds

{'01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14'}

In [6]:
initial_partition = Partition(
    graph,
    assignment="CD",
    updaters={
        "population": Tally("TOTPOP", alias="population"),
        "BVAP20": Tally("BVAP", alias="BVAP20"),
        "WVAP20": Tally("WVAP", alias="WVAP20"),
        "VAP20": Tally("VAP", alias="VAP20"),
    }
)

In [7]:
lst = []
for key in initial_partition["BVAP20"].keys():
    lst.append(initial_partition["BVAP20"][key]/initial_partition["VAP20"][key])

lst.sort()
lst

[0.06287556400405069,
 0.07842597904763401,
 0.12208630330911502,
 0.14342258670091887,
 0.16655029940611626,
 0.21771385876793567,
 0.23699274458743902,
 0.2790595128086165,
 0.2801391784133971,
 0.32500615214248363,
 0.4855155378424452,
 0.5251087582686081,
 0.5467670115029124,
 0.5555841271260975]

In [8]:
pop = sum(data["TOTPOP"] for node, data in initial_partition.graph.nodes(data=True))
bvap = sum(data["BVAP"] for node, data in initial_partition.graph.nodes(data=True))

In [17]:
bvap 

2072946

In [9]:
initial_partition

<Partition [14 parts]>

In [10]:
proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target= pop / 14,
    epsilon=0.01,
    node_repeats=2, 
)


chain = MarkovChain(
    proposal=proposal,
    initial_state=initial_partition,
    constraints=[contiguous],
    accept=always_accept,
    total_steps=10000,
)

In [11]:
def compute_percent_list(name, partition):
    keys = list(partition[name].keys())
    results = []
    for key in keys:
        results.append(partition[name][key] / partition["VAP20"][key])
    
    return {name: results}

In [12]:
initial_partition.BVAP20

{'12': 169050,
 '10': 122589,
 '13': 264345,
 '03': 111311,
 '09': 32845,
 '04': 275786,
 '06': 63904,
 '07': 81104,
 '05': 298162,
 '14': 39916,
 '02': 251197,
 '08': 145558,
 '01': 144600,
 '11': 72579}

In [13]:
results = []
for step, partition in enumerate(chain):
    if step > 8999:
        new_row = {"step": step - 8999}
        new_row.update(compute_percent_list("BVAP20", partition))
        new_row.update(compute_percent_list("WVAP20", partition))
        results.append(new_row)
    
    
    lst = [
        val / partition.VAP20[key]
        for key, val in partition.BVAP20.items() 
        if (val/partition.VAP20[key]) > 0.5]
    print(f"step: {step}, partition: {len(lst)}, lst: {lst}", end="\r")

results_df = pd.DataFrame(results)
results_df


/home/peter/Dropbox/MADLAB/Git_Repos/gerrytools-dev/.venv/lib/python3.11/site-packages/gerrychain/tree.py:427: BipartitionWarning: 
Failed to find a balanced cut after 50 attempts.
If possible, consider enabling pair reselection within your
MarkovChain proposal method to allow the algorithm to select
a different pair of districts to try and recombine.
  warnings.warn(


,step,BVAP20,WVAP20
0,1,"[0.796561916484547, 0.3328762902480357, 0.0451...","[0.6021837929440764, 0.5730542030111124, 0.627..."
1,2,"[0.37114006125403076, 0.3328762902480357, 0.04...","[0.6021837929440764, 0.5730542030111124, 0.627..."
2,3,"[0.37114006125403076, 0.3328762902480357, 0.04...","[0.6021837929440764, 0.5730542030111124, 0.627..."
3,4,"[0.37114006125403076, 0.3328762902480357, 0.04...","[0.6021837929440764, 0.5730542030111124, 0.627..."
4,5,"[0.37114006125403076, 0.3328762902480357, 0.08...","[0.6021837929440764, 0.5730542030111124, 0.627..."
...,...,...,...
995,996,"[0.1614882052740516, 0.3940210855076915, 0.350...","[0.39489255568342874, 0.6814721187538575, 0.55..."
996,997,"[0.1614882052740516, 0.3940210855076915, 0.350...","[0.39489255568342874, 0.6251441822104552, 0.55..."
997,998,"[0.1614882052740516, 0.3940210855076915, 0.350...","[0.39489255568342874, 0.6251441822104552, 0.55..."
998,999,"[0.1614882052740516, 0.6197015068299115, 0.350...","[0.27459097338459176, 0.6251441822104552, 0.55..."


In [14]:
results

[{'step': 1,
  'BVAP20': [0.796561916484547,
   0.3328762902480357,
   0.04510853849452073,
   0.3390029578096157,
   0.08184348252955288,
   0.36853268012904766,
   0.3758563067575734,
   0.34092315476074647,
   0.16481106078363056,
   0.3689490286314466,
   0.27755704812900817,
   0.2889411490321076,
   0.14496174336179848,
   0.12024937319496017],
  'WVAP20': [0.6021837929440764,
   0.5730542030111124,
   0.6271690118945814,
   0.5901055297924443,
   0.6871278230021222,
   0.8372431122677915,
   0.5234837945593848,
   0.3992002179454747,
   0.6271541464343521,
   0.11766183526338266,
   0.5035413578775346,
   0.6932558437804488,
   0.6489827843845339,
   0.8095174942002707]},
 {'step': 2,
  'BVAP20': [0.37114006125403076,
   0.3328762902480357,
   0.04510853849452073,
   0.3390029578096157,
   0.08184348252955288,
   0.36853268012904766,
   0.3758563067575734,
   0.34092315476074647,
   0.16481106078363056,
   0.7466969404510199,
   0.27755704812900817,
   0.2889411490321076,
   0.1

In [15]:
import json

In [16]:
with open("./data/ensemble_example.json", "w") as f:
    json.dump(results, f)